In [1]:
import pandas as pd
from sqlalchemy import create_engine

# Step 1: Extract


In [2]:
# Load the data into a Pandas DataFrame
file_path = 'data/all_data_M_2023.xlsx'
df = pd.read_excel(file_path, sheet_name='All May 2023 data')

In [3]:
# Load the "Field Descriptions" sheet to get the mappings
descriptions_df = pd.read_excel(file_path, sheet_name='Field Descriptions')

In [11]:
# Load the "Field Descriptions" sheet and process it
descriptions_df = pd.read_excel(file_path, sheet_name='Field Descriptions', skiprows=9)
descriptions_df.columns = ['Field', 'Field Description', 'Unused']
descriptions_df = descriptions_df.drop(columns=['Unused'])

# Display the first few rows to confirm the structure
descriptions_df.head()

,Field,Field Description
0,area,"U.S. (99), state FIPS code, Metropolitan Stati..."
1,area_title,Area name
2,area_type,Area type: 1= U.S.; 2= State; 3= U.S. Territor...
3,prim_state,"The primary state for the given area. ""US"" is ..."
4,naics,North American Industry Classification System ...


# Step 2: Transform the data

In [12]:
# Create a dictionary to map fields to descriptions
descriptions_dict = dict(zip(descriptions_df['Field'], descriptions_df['Field Description']))


In [14]:
# Define the columns we are keeping and map to their appropriate descriptions
columns_to_keep = ['AREA', 'AREA_TYPE', 'AREA_TITLE', 'PRIM_STATE', 'OCC_CODE', 'OCC_TITLE', 'O_GROUP', 
                   'TOT_EMP', 'EMP_PRSE', 'JOBS_1000', 'LOC_QUOTIENT', 'H_MEAN', 'A_MEAN', 
                   'MEAN_PRSE', 'H_PCT25', 'H_MEDIAN', 'H_PCT75', 'A_PCT25', 'A_MEDIAN', 'A_PCT75']

# Define the simplified column names as per the user's request
column_names_simple = [
    'area', 
    'area_type',
    'area_title', 
    'primary_state', 
    'occupation_code', 
    'occupation_title', 
    'occupation_group', 
    'total_employment', 
    'employment_prse', 
    'jobs_per_1000', 
    'location_quotient', 
    'mean_hourly_wage', 
    'mean_annual_wage', 
    'mean_prse', # Percent relative standard error (PRSE) for the mean wage estimate. 
    'hourly_percentile_25', 
    'hourly_median', 
    'hourly_percentile_75', 
    'annual_percentile_25', 
    'annual_median', 
    'annual_percentile_75'
]

renamed_columns = {col: descriptions_dict.get(col.lower(), col) for col in columns_to_keep}  

In [15]:
# Keep only the relevant columns and rename them based on the descriptions
df_cleaned = df[columns_to_keep].dropna(subset=columns_to_keep)
df_cleaned = df_cleaned.rename(columns=renamed_columns)
df_cleaned.head()

,"U.S. (99), state FIPS code, Metropolitan Statistical Area (MSA) or New England City and Town Area (NECTA) code, or OEWS-specific nonmetropolitan area code",Area type: 1= U.S.; 2= State; 3= U.S. Territory; 4= Metropolitan Statistical Area (MSA) or New England City and Town Area (NECTA); 6= Nonmetropolitan Area,Area name,"The primary state for the given area. ""US"" is used for the national estimates.",The 6-digit Standard Occupational Classification (SOC) code or OEWS-specific code for the occupation,SOC title or OEWS-specific title for the occupation,"SOC occupation level. For most occupations, this field indicates the standard SOC major, minor, broad, and detailed levels, in addition to all-occupations totals. For occupations that OEWS no longer publishes at the SOC detailed level, the “detailed” designation indicates the most detailed data available: either a standard SOC broad occupation or an OEWS-specific combination of detailed occupations. Occupations that OEWS has aggregated to the SOC broad occupation level will appear in the file twice, once with the “broad” and once with the “detailed” designation.",Estimated total employment rounded to the nearest 10 (excludes self-employed).,"Percent relative standard error (PRSE) for the employment estimate. PRSE is a measure of sampling error, expressed as a percentage of the corresponding estimate. Sampling error occurs when values for a population are estimated from a sample survey of the population, rather than calculated from data for all members of the population. Estimates with lower PRSEs are typically more precise in the presence of sampling error.","The number of jobs (employment) in the given occupation per 1,000 jobs in the given area. Only available for the state and MSA estimates; otherwise, this column is blank.",LOC_QUOTIENT,Mean hourly wage,Mean annual wage,"Percent relative standard error (PRSE) for the mean wage estimate. PRSE is a measure of sampling error, expressed as a percentage of the corresponding estimate. Sampling error occurs when values for a population are estimated from a sample survey of the population, rather than calculated from data for all members of the population. Estimates with lower PRSEs are typically more precise in the presence of sampling error.",Hourly 25th percentile wage,Hourly median wage (or the 50th percentile),Hourly 75th percentile wage,Annual 25th percentile wage,Annual median wage (or the 50th percentile),Annual 75th percentile wage
177501,1,2,Alabama,AL,00-0000,All Occupations,total,2053090,0,1000,1,25.67,53400,0.2,14.22,19.88,30.09,29580,41350,62580
177502,2,2,Alaska,AK,00-0000,All Occupations,total,311610,0,1000,1,33.6,69880,0.3,18.62,26.99,40.52,38720,56140,84280
177503,4,2,Arizona,AZ,00-0000,All Occupations,total,3129720,0,1000,1,30.31,63040,0.5,17.38,22.92,35.05,36150,47680,72900
177504,5,2,Arkansas,AR,00-0000,All Occupations,total,1271320,0,1000,1,24.64,51250,0.2,14.42,18.78,28.32,30000,39060,58900
177505,6,2,California,CA,00-0000,All Occupations,total,17945910,0,1000,1,37,76960,0.4,18.21,25.98,44.83,37890,54030,93250


In [16]:
# Map the original column names to simplified names
renamed_columns = dict(zip(columns_to_keep, column_names_simple))

# Keep only the relevant columns and rename them based on the simplified names
df_cleaned = df[columns_to_keep].dropna(subset=columns_to_keep)
df_cleaned = df_cleaned.rename(columns=renamed_columns)

# Display the first few rows to confirm the renaming
df_cleaned.head()


,area,area_type,area_title,primary_state,occupation_code,occupation_title,occupation_group,total_employment,employment_prse,jobs_per_1000,location_quotient,mean_hourly_wage,mean_annual_wage,mean_prse,hourly_percentile_25,hourly_median,hourly_percentile_75,annual_percentile_25,annual_median,annual_percentile_75
177501,1,2,Alabama,AL,00-0000,All Occupations,total,2053090,0,1000,1,25.67,53400,0.2,14.22,19.88,30.09,29580,41350,62580
177502,2,2,Alaska,AK,00-0000,All Occupations,total,311610,0,1000,1,33.6,69880,0.3,18.62,26.99,40.52,38720,56140,84280
177503,4,2,Arizona,AZ,00-0000,All Occupations,total,3129720,0,1000,1,30.31,63040,0.5,17.38,22.92,35.05,36150,47680,72900
177504,5,2,Arkansas,AR,00-0000,All Occupations,total,1271320,0,1000,1,24.64,51250,0.2,14.42,18.78,28.32,30000,39060,58900
177505,6,2,California,CA,00-0000,All Occupations,total,17945910,0,1000,1,37,76960,0.4,18.21,25.98,44.83,37890,54030,93250


In [18]:
state_oews_df = df_cleaned.loc[df_cleaned['area_type'] == 2, :]
state_oews_df

,area,area_type,area_title,primary_state,occupation_code,occupation_title,occupation_group,total_employment,employment_prse,jobs_per_1000,location_quotient,mean_hourly_wage,mean_annual_wage,mean_prse,hourly_percentile_25,hourly_median,hourly_percentile_75,annual_percentile_25,annual_median,annual_percentile_75
177501,1,2,Alabama,AL,00-0000,All Occupations,total,2053090,0,1000,1,25.67,53400,0.2,14.22,19.88,30.09,29580,41350,62580
177502,2,2,Alaska,AK,00-0000,All Occupations,total,311610,0,1000,1,33.6,69880,0.3,18.62,26.99,40.52,38720,56140,84280
177503,4,2,Arizona,AZ,00-0000,All Occupations,total,3129720,0,1000,1,30.31,63040,0.5,17.38,22.92,35.05,36150,47680,72900
177504,5,2,Arkansas,AR,00-0000,All Occupations,total,1271320,0,1000,1,24.64,51250,0.2,14.42,18.78,28.32,30000,39060,58900
177505,6,2,California,CA,00-0000,All Occupations,total,17945910,0,1000,1,37,76960,0.4,18.21,25.98,44.83,37890,54030,93250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214139,49,2,Utah,UT,53-7199,"Material Moving Workers, All Other",detailed,160,26,0.093,0.59,21.18,44050,5.8,17,20.81,23.38,35360,43280,48630
214140,51,2,Virginia,VA,53-7199,"Material Moving Workers, All Other",detailed,270,20.2,0.069,0.44,20.25,42110,4.1,15.37,18.21,24.64,31980,37880,51240
214141,53,2,Washington,WA,53-7199,"Material Moving Workers, All Other",detailed,100,16.1,0.028,0.18,30.13,62660,5,22.07,26.9,35.69,45900,55960,74240
214142,54,2,West Virginia,WV,53-7199,"Material Moving Workers, All Other",detailed,30,42,0.05,0.32,15.98,33250,4.5,13.53,15.12,15.92,28140,31440,33110


In [19]:
# Convert object columns to numeric
columns_to_convert = [
    'total_employment', 
    'employment_prse', 
    'jobs_per_1000', 
    'location_quotient', 
    'mean_hourly_wage', 
    'mean_annual_wage', 
    'mean_prse', 
    'hourly_percentile_25', 
    'hourly_median', 
    'hourly_percentile_75', 
    'annual_percentile_25', 
    'annual_median', 
    'annual_percentile_75'
]

In [20]:
# Convert the specified columns to numeric, coercing errors
df_cleaned[columns_to_convert] = df_cleaned[columns_to_convert].apply(pd.to_numeric, errors='coerce')

df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 235826 entries, 177501 to 413326
Data columns (total 20 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   area                  235826 non-null  int64  
 1   area_type             235826 non-null  int64  
 2   area_title            235826 non-null  object 
 3   primary_state         235826 non-null  object 
 4   occupation_code       235826 non-null  object 
 5   occupation_title      235826 non-null  object 
 6   occupation_group      235826 non-null  object 
 7   total_employment      232193 non-null  float64
 8   employment_prse       232193 non-null  float64
 9   jobs_per_1000         232193 non-null  float64
 10  location_quotient     232193 non-null  float64
 11  mean_hourly_wage      220341 non-null  float64
 12  mean_annual_wage      233028 non-null  float64
 13  mean_prse             233611 non-null  float64
 14  hourly_percentile_25  219933 non-null  float64
 15  

In [22]:
df_cleaned = df_cleaned[[
    'area', 
    'area_title', 
    'primary_state', 
    'occupation_code', 
    'occupation_title', 
    'occupation_group', 
    'total_employment', 
    'employment_prse', 
    'jobs_per_1000', 
    'location_quotient', 
    'mean_hourly_wage', 
    'mean_annual_wage', 
    'mean_prse', # Percent relative standard error (PRSE) for the mean wage estimate. 
    'hourly_percentile_25', 
    'hourly_median', 
    'hourly_percentile_75', 
    'annual_percentile_25', 
    'annual_median', 
    'annual_percentile_75'
]]
df_cleaned.head()

,area,area_title,primary_state,occupation_code,occupation_title,occupation_group,total_employment,employment_prse,jobs_per_1000,location_quotient,mean_hourly_wage,mean_annual_wage,mean_prse,hourly_percentile_25,hourly_median,hourly_percentile_75,annual_percentile_25,annual_median,annual_percentile_75
177501,1,Alabama,AL,00-0000,All Occupations,total,2053090.0,0.0,1000.0,1.0,25.67,53400.0,0.2,14.22,19.88,30.09,29580.0,41350.0,62580.0
177502,2,Alaska,AK,00-0000,All Occupations,total,311610.0,0.0,1000.0,1.0,33.60,69880.0,0.3,18.62,26.99,40.52,38720.0,56140.0,84280.0
177503,4,Arizona,AZ,00-0000,All Occupations,total,3129720.0,0.0,1000.0,1.0,30.31,63040.0,0.5,17.38,22.92,35.05,36150.0,47680.0,72900.0
177504,5,Arkansas,AR,00-0000,All Occupations,total,1271320.0,0.0,1000.0,1.0,24.64,51250.0,0.2,14.42,18.78,28.32,30000.0,39060.0,58900.0
177505,6,California,CA,00-0000,All Occupations,total,17945910.0,0.0,1000.0,1.0,37.00,76960.0,0.4,18.21,25.98,44.83,37890.0,54030.0,93250.0


In [24]:
occ_code_list = ['00-0000', '11-0000', '13-0000', '15-0000', '17-0000', '19-0000', '21-0000', '23-0000', '25-0000', '27-0000', '29-0000', '31-0000', '33-0000',\
    '35-0000', '37-0000', '39-0000', '41-0000', '43-0000', '45-0000', '47-0000', '49-0000', '51-0000', '53-0000', '11-3021', '13-1161', '13-2031', '13-2041',\
        '13-2054', '15-1221', '15-1251', '15-1252', '15-1253', '15-1254', '15-2011', '15-2031', '15-2041', '19-1022', '19-1029', '19-2041', '29-9091', '41-3021',\
            '43-9021', '13-2051','15-1211', '15-1242', '15-1243', '15-2051']

df_filtered = df_cleaned[df_cleaned['occupation_code'].isin(occ_code_list)]
df_filtered

,area,area_title,primary_state,occupation_code,occupation_title,occupation_group,total_employment,employment_prse,jobs_per_1000,location_quotient,mean_hourly_wage,mean_annual_wage,mean_prse,hourly_percentile_25,hourly_median,hourly_percentile_75,annual_percentile_25,annual_median,annual_percentile_75
177501,1,Alabama,AL,00-0000,All Occupations,total,2053090.0,0.0,1000.000,1.00,25.67,53400.0,0.2,14.22,19.88,30.09,29580.0,41350.0,62580.0
177502,2,Alaska,AK,00-0000,All Occupations,total,311610.0,0.0,1000.000,1.00,33.60,69880.0,0.3,18.62,26.99,40.52,38720.0,56140.0,84280.0
177503,4,Arizona,AZ,00-0000,All Occupations,total,3129720.0,0.0,1000.000,1.00,30.31,63040.0,0.5,17.38,22.92,35.05,36150.0,47680.0,72900.0
177504,5,Arkansas,AR,00-0000,All Occupations,total,1271320.0,0.0,1000.000,1.00,24.64,51250.0,0.2,14.42,18.78,28.32,30000.0,39060.0,58900.0
177505,6,California,CA,00-0000,All Occupations,total,17945910.0,0.0,1000.000,1.00,37.00,76960.0,0.4,18.21,25.98,44.83,37890.0,54030.0,93250.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
411030,5600006,Western Wyoming nonmetropolitan area,WY,53-0000,Transportation and Material Moving Occupations,major,7560.0,1.9,75.949,0.84,24.22,50380.0,1.7,17.16,21.62,28.21,35690.0,44980.0,58670.0
411031,5600007,Eastern Wyoming nonmetropolitan area,WY,53-0000,Transportation and Material Moving Occupations,major,6980.0,2.1,83.274,0.92,23.58,49050.0,1.4,16.87,20.53,28.47,35080.0,42690.0,59210.0
411032,6600001,Guam,GU,53-0000,Transportation and Material Moving Occupations,major,4450.0,3.2,72.648,0.80,17.43,36250.0,2.3,10.24,11.32,17.20,21290.0,23550.0,35780.0
411033,7200006,Puerto Rico nonmetropolitan area,PR,53-0000,Transportation and Material Moving Occupations,major,1550.0,2.3,75.648,0.84,11.32,23550.0,0.9,9.06,10.09,12.59,18850.0,20990.0,26180.0


# Step 3: Load the Data

In [28]:
# Export the data to CSV
df_filtered.to_csv('data/EmploymentWageData.csv', index=False)